In [1]:
import pandas as pd
import os
import oracledb
from pandas import read_csv

In [2]:
# Create DSN (Data Source Name)
dsn = oracledb.makedsn("localhost", 1521, service_name="XEPDB1")

# Connect
conn = oracledb.connect(
    user="Ecommerce",
    password="Bhimd123",
    dsn=dsn
)

cursor = conn.cursor()
print("Connected to Oracle Database")

Connected to Oracle Database


folder location csv file name

In [26]:
# Folder containing the CSV files
folder_path = 'Original_data'
# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('geolocation.csv', 'geolocation'),
    ('order_items.csv', 'order_items'),
    ('orders.csv', 'orders'),
    ('products.csv', 'products'),
    ('payments.csv', 'payments'),
    ('sellers.csv', 'sellers'),# Added payments.csv for specific handling
]

Datatype of finding function

In [27]:
def get_sql_type(dtype, sample_values=None):
    if pd.api.types.is_integer_dtype(dtype):
        return "NUMBER"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATE"
    else:
        # If object/string, pick VARCHAR2 with max length from sample
        max_len = 255
        if sample_values is not None:
            try:
                max_len = max(sample_values.dropna().astype(str).map(len).max(), 1)
            except:
                pass
        return f"VARCHAR2({min(max_len, 4000)})"  # Oracle limit is 4000 for VARCHAR2


In [29]:
# ---------- STEP 3: Process each CSV ----------
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    # Read CSV
    df = pd.read_csv(file_path, delimiter=',', header=0)

    # Replace NaN with None
    df = df.where(pd.notnull(df), None)

    print(f"\nProcessing {csv_file}")
    print("Dtypes:\n", df.dtypes)
    print("NaN values per column:\n", df.isnull().sum())

    # Clean column names
    df.columns = [col.upper().replace(" ", "_").replace("-", "_").replace(".", "_") for col in df.columns]

    # ---------- STEP 4: Drop existing table (if any) ----------
    try:
        cursor.execute(f"DROP TABLE {table_name}")
        print(f"Table {table_name} dropped.")
    except oracledb.DatabaseError:
        print(f"Table {table_name} does not exist, creating new one.")

    # ---------- STEP 5: Build CREATE TABLE ----------
    column_defs = []
    for col in df.columns:
        sql_type = get_sql_type(df[col].dtype, df[col])
        column_defs.append(f"{col} {sql_type}")

    create_table_query = f"CREATE TABLE {table_name} ({', '.join(column_defs)})"
    print("Executing:", create_table_query)
    cursor.execute(create_table_query)

    # # ---------- STEP 6: Insert rows ----------
    # I have Inserted rows in Table using import data.
    # placeholders = ", ".join([f"{i}:{i+1}" for i in range(len(df.columns))])
    # insert_sql = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"

    # for _, row in df.iterrows():
    #     values = tuple(None if pd.isna(x) else x for x in row)
    #     cursor.execute(insert_sql, values)

    # conn.commit()
    # print(f"Inserted {len(df)} rows into {table_name}.")

# ---------- STEP 7: Close connection ----------
conn.close()
print("\nAll CSVs processed successfully.")


Processing customers.csv
Dtypes:
 customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object
NaN values per column:
 customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
Table customers dropped.
Executing: CREATE TABLE customers (CUSTOMER_ID VARCHAR2(32), CUSTOMER_UNIQUE_ID VARCHAR2(32), CUSTOMER_ZIP_CODE_PREFIX NUMBER, CUSTOMER_CITY VARCHAR2(32), CUSTOMER_STATE VARCHAR2(2))

Processing geolocation.csv
Dtypes:
 geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object
NaN values per column:
 geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_